In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
rdd1 = sc.parallelize([1,2,3,4,5,6])

In [3]:
rdd1.first()

1

In [4]:
rdd1.count()

6

In [5]:
rdd1.collect()

[1, 2, 3, 4, 5, 6]

In [6]:
rdd2 = rdd1.cache()

In [7]:
rdd2.collect()

[1, 2, 3, 4, 5, 6]

In [8]:
rdd2.max()

6

In [9]:
rdd2.mean()

3.5

In [10]:
rdd2.min()

1

In [11]:
rdd2.sum()

21

In [12]:
rdd2.filter(lambda x: x > 4).collect()

[5, 6]

In [13]:
rdd2.collect()

[1, 2, 3, 4, 5, 6]

In [14]:
dfAluno = spark.read.option("header",True).format('csv').load('notas.csv')

In [15]:
dfAluno.collect()

[Row(aluno='jean', nota='8', disciplina=' matemática'),
 Row(aluno='joão', nota='6', disciplina=' português'),
 Row(aluno='jean', nota='5', disciplina=' português'),
 Row(aluno='joão', nota='5', disciplina=' matemática'),
 Row(aluno='caio', nota='5', disciplina=' português'),
 Row(aluno='caio', nota='3', disciplina=' matemática')]

In [16]:
type(dfAluno)

pyspark.sql.dataframe.DataFrame

In [17]:
dfAluno.first()

Row(aluno='jean', nota='8', disciplina=' matemática')

In [18]:
dfAluno.filter(dfAluno.aluno == 'jean').show()

+-----+----+-----------+
|aluno|nota| disciplina|
+-----+----+-----------+
| jean|   8| matemática|
| jean|   5|  português|
+-----+----+-----------+



In [19]:
dfAluno.filter((dfAluno.aluno == 'jean') & (dfAluno.nota == 8)).show()

+-----+----+-----------+
|aluno|nota| disciplina|
+-----+----+-----------+
| jean|   8| matemática|
+-----+----+-----------+



In [20]:
dfAluno.filter(dfAluno.aluno == 'jean').select('nota','disciplina').show()

+----+-----------+
|nota| disciplina|
+----+-----------+
|   8| matemática|
|   5|  português|
+----+-----------+



In [21]:
dfAluno.createOrReplaceTempView('notas')

In [22]:
spark.sql('select * from notas').show()

+-----+----+-----------+
|aluno|nota| disciplina|
+-----+----+-----------+
| jean|   8| matemática|
| joão|   6|  português|
| jean|   5|  português|
| joão|   5| matemática|
| caio|   5|  português|
| caio|   3| matemática|
+-----+----+-----------+



In [23]:
spark.sql("select * from notas where aluno == 'caio'").show()

+-----+----+-----------+
|aluno|nota| disciplina|
+-----+----+-----------+
| caio|   5|  português|
| caio|   3| matemática|
+-----+----+-----------+



In [24]:
dfAluno.groupBy('aluno').sum('nota').show()

AnalysisException: '"nota" is not a numeric column. Aggregation function can only be applied on a numeric column.;'

In [25]:
dfAluno.describe()

DataFrame[summary: string, aluno: string, nota: string, disciplina: string]

In [26]:
dfAluno.select(dfAluno.nota.cast('int'),dfAluno.aluno).groupBy('aluno').sum('nota').withColumnRenamed("sum(nota)", "nota").show()

+-----+----+
|aluno|nota|
+-----+----+
| caio|   8|
| jean|  13|
| joão|  11|
+-----+----+



In [27]:
dfAluno.show()

+-----+----+-----------+
|aluno|nota| disciplina|
+-----+----+-----------+
| jean|   8| matemática|
| joão|   6|  português|
| jean|   5|  português|
| joão|   5| matemática|
| caio|   5|  português|
| caio|   3| matemática|
+-----+----+-----------+



In [28]:
from pyspark.sql.functions import col

In [29]:
dfAluno = dfAluno.withColumn('nota2', col("nota") * 100)

In [30]:
dfAluno.show()

+-----+----+-----------+-----+
|aluno|nota| disciplina|nota2|
+-----+----+-----------+-----+
| jean|   8| matemática|800.0|
| joão|   6|  português|600.0|
| jean|   5|  português|500.0|
| joão|   5| matemática|500.0|
| caio|   5|  português|500.0|
| caio|   3| matemática|300.0|
+-----+----+-----------+-----+



In [31]:
text_file = sc.textFile("notas.csv")
counts = text_file.flatMap(lambda line: line.split(",")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

In [32]:
counts.collect()

[('disciplina', 1),
 ('8', 1),
 (' matemática', 3),
 ('joão', 2),
 (' português', 3),
 ('aluno', 1),
 ('nota', 1),
 ('jean', 2),
 ('6', 1),
 ('5', 3),
 ('caio', 2),
 ('3', 1)]

#### https://spark.apache.org/examples.html
#### https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf